<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Requirements installation

In [1]:
!pip install sagemaker --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade boto3 --no-cache | tail -n 1
!pip install --upgrade pandas --no-cache | tail -n 1

Requirement not upgraded as not directly required: docker-pycreds>=0.4.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from docker<4.0,>=3.6.0->docker-compose>=1.23.0->sagemaker)
Requirement not upgraded as not directly required: docutils>=0.10 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client->ibm-ai-openscale)
Requirement not upgraded as not directly required: six>=1.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from python-dateutil<3.0.0,>=2.1; python_version >= "2.7"->botocore<1.13.0,>=1.12.94->boto3)
Requirement not upgraded as not directly required: six>=1.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from python-dateutil>=2.5.0->pandas)


**Action:** Restart the kernel.

### 1.1 Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- You will create SageMaker model using notebook from [github repository](https://github.com/pmservice/ai-openscale-tutorials/blob/master/notebooks/Credit%20%20model%20with%20SageMaker%20linear-learner%20.ipynb)
- Run the code to train a SageMaker model and create deployment endpoint for online inference

### 1.2 Authentication

Import and initiate.

In [2]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```bash
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```bash
ibmcloud resource service-instance 'AI-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [3]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

Create schema for data mart.

In [8]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [9]:
client = APIClient(AIOS_CREDENTIALS)

In [10]:
client.version

'2.0.18'

### 1.3 DataMart setup

In [11]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

Warning during setup of data mart.
Status code: 409, body: {"trace":"Mjc4YTFhY2YtNTRiOS00NWZiLTg1ZDgtYjMyMjQwOGI5YjQ4","errors":[{"code":"AISCS0004W","message":"Data Mart with this id is already defined"}]}


In [12]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [13]:
SAGEMAKER_ENGINE_CREDENTIALS = {
        "access_key_id": "***",
        "secret_access_key": "***",
        "region": "***"
}

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
binding_uid = client.data_mart.bindings.add('My SageMaker engine', SageMakerMachineLearningInstance(SAGEMAKER_ENGINE_CREDENTIALS))

In [16]:
bindings_details = client.data_mart.bindings.get_details()

In [17]:
client.data_mart.bindings.list()

988e95d1-66f2-4911-9cfd-65eea194b7ab,My SageMaker engine,amazon_sagemaker,2019-02-14T15:29:24.028Z
d47e97b7-07d7-4874-b604-e47ff1a0cd26,My SageMaker engine,amazon_sagemaker,2019-02-14T12:50:02.856Z
a73bf76d-a663-448a-b771-4f651f73d54e,WML instance,watson_machine_learning,2019-02-13T10:34:55.751Z
91d749e3-8551-45da-84fc-385e7ed2ba26,My Azure ML Studio engine,azure_machine_learning,2019-02-07T12:44:20.125Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [18]:
client.data_mart.bindings.list_assets()

57d1b9ccf254e8f05fdcb0ac05744e94,Credit-risk-linear-learner-2019-02-14-14-05,2019-02-14T14:11:25.761Z,model,,988e95d1-66f2-4911-9cfd-65eea194b7ab,False
57d1b9ccf254e8f05fdcb0ac05744e94,Credit-risk-linear-learner-2019-02-14-14-05,2019-02-14T14:11:25.761Z,model,,d47e97b7-07d7-4874-b604-e47ff1a0cd26,False
e23cada2cc488830e5dfc26172bdf58d,Credit-risk-linear-learner-2019-02-14-13-28,2019-02-14T13:37:28.847Z,model,,988e95d1-66f2-4911-9cfd-65eea194b7ab,False
e23cada2cc488830e5dfc26172bdf58d,Credit-risk-linear-learner-2019-02-14-13-28,2019-02-14T13:37:28.847Z,model,,d47e97b7-07d7-4874-b604-e47ff1a0cd26,False
9dc24903-f6b7-4ddb-8239-3d30249e3ea7,AIOS Xgboost Agaricus Model,2019-02-13T12:17:41.412Z,model,xgboost-0.80,a73bf76d-a663-448a-b771-4f651f73d54e,False
2ac283ef-5138-47d6-9743-d4b230e3f056,AIOS Xgboost Agaricus Model,2019-02-13T12:14:41.486Z,model,xgboost-0.80,a73bf76d-a663-448a-b771-4f651f73d54e,False
c02dbfe4-ad80-45ec-81b8-ea5fb7b33699,AIOS Xgboost Agaricus Model,2019-02-13T10:59:51.344Z,model,xgboost-0.80,a73bf76d-a663-448a-b771-4f651f73d54e,False
0f1d511c-b121-44d1-b3f7-c3ad258211f5,AIOS Spark German Risk Model - Final,2019-02-13T10:34:05.152Z,model,mllib-2.1,a73bf76d-a663-448a-b771-4f651f73d54e,True
7d56bf771bfcd6d9b7d9a58bf3fdf24d,ExecutePythonScr.2019.2.13.8.6.20.493,2019-02-13T08:06:27.6046787Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
3d9d326a906661b64ab9e3f6b232189b,hung-TensorflowMNISTE.2019.2.13.7.18.28.543,2019-02-13T07:18:46.3150458Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
b9abec257bde66c506a9412bca56bee5,hung-sklearnexperimen.2019.2.13.2.21.5.502,2019-02-13T02:21:29.6267608Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False


**Action:** Assign your source_uid to `source_uid` variable below.

In [19]:
source_uid = '57d1b9ccf254e8f05fdcb0ac05744e94'

In [20]:
subscription = client.data_mart.subscriptions.add(
    SageMakerMachineLearningAsset(
                source_uid=source_uid,
                binding_uid=binding_uid,
                input_data_type=InputDataType.STRUCTURED,
                problem_type=ProblemType.BINARY_CLASSIFICATION,
                label_column='Risk',
                probability_column='score',
                prediction_column='predicted_label'))

#### Get subscriptions list

In [21]:
subscriptions = client.data_mart.subscriptions.get_details()

In [22]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['085460ef94636166aea5800e9ea26168', '57d1b9ccf254e8f05fdcb0ac05744e94', '0f1d511c-b121-44d1-b3f7-c3ad258211f5', '0530ab0cd4f4dd5486b19c08df8b6914']


#### List subscriptions

In [23]:
client.data_mart.subscriptions.list()

57d1b9ccf254e8f05fdcb0ac05744e94,Credit-risk-linear-learner-2019-02-14-14-05,model,988e95d1-66f2-4911-9cfd-65eea194b7ab,2019-02-14T15:31:32.264Z
0530ab0cd4f4dd5486b19c08df8b6914,DEMO-multi-classification-2018-10-10-14-26-26,model,d47e97b7-07d7-4874-b604-e47ff1a0cd26,2019-02-14T12:50:55.426Z
0f1d511c-b121-44d1-b3f7-c3ad258211f5,AIOS Spark German Risk Model - Final,model,a73bf76d-a663-448a-b771-4f651f73d54e,2019-02-13T10:36:38.050Z
085460ef94636166aea5800e9ea26168,GermanCreditRisk.2019.1.9.10.41.58.611,model,91d749e3-8551-45da-84fc-385e7ed2ba26,2019-02-07T12:46:42.998Z


<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the product line model and measure response time

In [24]:
import requests
import time
import json
import boto3

binding_details = client.data_mart.bindings.get_details(binding_uid=binding_uid)
subscription_details = subscription.get_details()

access_id = binding_details['entity']['credentials']['access_key_id']
access_key = binding_details['entity']['credentials']['secret_access_key']
region = binding_details['entity']['credentials']['region']
endpoint_name = subscription_details['entity']['deployments'][0]['name']
runtime = boto3.client('sagemaker-runtime', region_name=region, aws_access_key_id=access_id, aws_secret_access_key=access_key)

fields = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes', 'Risk_No Risk', 'Risk_Risk', 'LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents']
        
payload = "0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,18,462,2,2,37,2,1"

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": 0.00043791052303276956,
      "predicted_label": 0.0
    }
  ]
}


### 4.2 Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [25]:
values = []

for v in payload.split('\n'):
    values.append([float(s) for s in v.split(',')])

request_data = {'fields': fields, 'values': values}

response_data = {'fields': list(result['predictions'][0]),
            'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [26]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

#### Store the payload using REST API

Get the token first.

In [27]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":AIOS_CREDENTIALS["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

Store the payload.

In [28]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(AIOS_CREDENTIALS['url'], AIOS_CREDENTIALS['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

Request OK: True


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [29]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=5)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [31]:
subscription.feedback_logging.print_table_schema()

CheckingStatus_0_to_200,double,True
CheckingStatus_greater_200,double,True
CheckingStatus_less_0,double,True
CheckingStatus_no_checking,double,True
CreditHistory_all_credits_paid_back,double,True
CreditHistory_credits_paid_to_date,double,True
CreditHistory_no_credits,double,True
CreditHistory_outstanding_credit,double,True
CreditHistory_prior_payments_delayed,double,True
LoanPurpose_appliances,double,True
LoanPurpose_business,double,True


The feedback records can be send to feedback table using below code.

In [51]:
feedback_records = ['0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,18,462,2,2,37,2,1,0']

fields = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes', 'Risk_No Risk', 'Risk_Risk', 'LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents','Risk']

for i in range(1, 10):
    feedback_records.append('0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,18,462,2,2,37,2,1,0')

subscription.feedback_logging.store(feedback_data='\n'.join(feedback_records), fields=fields, feedback_format='text')

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [52]:
run_details = subscription.quality_monitoring.run()

Since the monitoring runs in the background you can use below method to check the status of the job.

In [53]:
status = run_details['status']
id = run_details['id']

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

Run status: initializing
Run status: completed


### Show the quality metrics

In [54]:
subscription.quality_monitoring.show_table()

2019-02-14 15:54:35.014000+00:00,0.0,0.7,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,Accuracy_evaluation_03bfa81b-299b-414b-b779-a3e1be9f22da,


Get all calculated metrics.

In [55]:
deployment_uids = subscription.get_deployment_uids()

In [56]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

{'end': '2019-02-14T15:54:54.458360Z',
 'metrics': [{'process': 'Accuracy_evaluation_03bfa81b-299b-414b-b779-a3e1be9f22da',
   'timestamp': '2019-02-14T15:54:35.014Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 0.0},
     {'name': 'areaUnderPR', 'value': 0.0}],
    'quality': 0.0,
    'threshold': 0.7}}],
 'start': '2019-02-14T14:31:32.264Z'}

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Print schema of payload_logging table

In [57]:
subscription.payload_logging.print_table_schema()

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
CheckingStatus_0_to_200,double,True
CheckingStatus_greater_200,double,True
CheckingStatus_less_0,double,True
CheckingStatus_no_checking,double,True
CreditHistory_all_credits_paid_back,double,True
CreditHistory_credits_paid_to_date,double,True
CreditHistory_no_credits,double,True


#### Show (preview) the table

In [58]:
subscription.payload_logging.describe_table()

       CheckingStatus_0_to_200  CheckingStatus_greater_200  \
count                      8.0                         8.0   
mean                       0.0                         0.0   
std                        0.0                         0.0   
min                        0.0                         0.0   
25%                        0.0                         0.0   
50%                        0.0                         0.0   
75%                        0.0                         0.0   
max                        0.0                         0.0   

       CheckingStatus_less_0  CheckingStatus_no_checking  \
count                    8.0                         8.0   
mean                     1.0                         0.0   
std                      0.0                         0.0   
min                      1.0                         0.0   
25%                      1.0                         0.0   
50%                      1.0                         0.0   
75%                  

#### Return the table content as pandas dataframe

In [59]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feddback logging

Check the schema of table.

In [60]:
subscription.feedback_logging.print_table_schema()

CheckingStatus_0_to_200,double,True
CheckingStatus_greater_200,double,True
CheckingStatus_less_0,double,True
CheckingStatus_no_checking,double,True
CreditHistory_all_credits_paid_back,double,True
CreditHistory_credits_paid_to_date,double,True
CreditHistory_no_credits,double,True
CreditHistory_outstanding_credit,double,True
CreditHistory_prior_payments_delayed,double,True
LoanPurpose_appliances,double,True
LoanPurpose_business,double,True


Preview table content.

In [61]:
subscription.feedback_logging.show_table()

0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,462.0,2.0,2.0,37.0,2.0,1.0,0.0,2019-02-14 15:54:26.909000+00:00


Describe table (calulcate basic statistics).

In [62]:
subscription.feedback_logging.describe_table()

       CheckingStatus_0_to_200  CheckingStatus_greater_200  \
count                     10.0                        10.0   
mean                       0.0                         0.0   
std                        0.0                         0.0   
min                        0.0                         0.0   
25%                        0.0                         0.0   
50%                        0.0                         0.0   
75%                        0.0                         0.0   
max                        0.0                         0.0   

       CheckingStatus_less_0  CheckingStatus_no_checking  \
count                   10.0                        10.0   
mean                     1.0                         0.0   
std                      0.0                         0.0   
min                      1.0                         0.0   
25%                      1.0                         0.0   
50%                      1.0                         0.0   
75%                  

Get table content.

In [63]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [64]:
subscription.quality_monitoring.print_table_schema()

ts,timestamp,False
quality,float,False
quality_threshold,float,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [65]:
subscription.quality_monitoring.show_table()

2019-02-14 15:54:35.014000+00:00,0.0,0.7,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,Accuracy_evaluation_03bfa81b-299b-414b-b779-a3e1be9f22da,


### 6.4 Performance metrics table

In [66]:
subscription.performance_monitoring.print_table_schema()

ts,timestamp,False
scoring_time,float,False
scoring_records,object,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [67]:
subscription.performance_monitoring.show_table()

2019-02-14 15:34:37.424655+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424708+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424796+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424812+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424730+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424828+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424764+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424860+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424748+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,
2019-02-14 15:34:37.424780+00:00,266.0,1,988e95d1-66f2-4911-9cfd-65eea194b7ab,57d1b9ccf254e8f05fdcb0ac05744e94,6722adccc8ad2c86d61fe46f32c4c03d,,


### 6.5 Data Mart measurement facts table

In [68]:
client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'asset': {'asset_id': '085460ef94636166aea5800e9ea26168',
    'asset_type': 'model',
    'created_at': '2019-01-09T10:42:59.7933412Z',
    'name': 'GermanCreditRisk.2019.1.9.10.41.58.611',
    'url': 'https://ussouthcentral.services.azureml.net/subscriptions/744bca722299451cb682ed6fb75fb671/services/e13d36b1c48f4080a49e5ae675d816ec/swagger.json'},
   'deployment': {'created_at': '2019-01-09T10:42:59.7933412Z',
    'deployment_id': '563f01d37f720857b95c557dc76176ad',
    'deployment_rn': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/ai-ops-squad/providers/Microsoft.MachineLearning/webServices/GermanCreditRisk.2019.1.9.10.41.58.611',
    'deployment_type': 'online',
    'name': 'GermanCreditRisk.2019.1.9.10.41.58.611',
    'scoring_endpoint': {'credentials': {'token': 'X/YNeesRdD95FDyXoBm7wvP2LlHD4pZGlMARZv6rh8AfV0Ol0Zb1KiftUJiNmisml7NmhURSpzeGn2UsKzcqZw=='},
     'request_headers': {'Content-Type': 'application/json'},
     'url': 'https://

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.